# Keras Intro: Time Series Forecasting

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Time series forecasting

In [ ]:
df = pd.read_csv('../data/ZonalDemands_2003-2016.csv',
                 engine='python')
df.head()

In [ ]:
df['Datetime'] = pd.to_datetime(df['Date']) + df['Hour'].apply(lambda h: pd.Timedelta("%d hours" % h))

In [ ]:
df = df.set_index('Datetime')

In [ ]:
from pandas.tseries.offsets import MonthEnd

In [ ]:
df['Total Ontario'].plot(figsize=(17,5))

In [ ]:
df['2015-01-01':'2015-12-31']['Total Ontario'].plot(style='p-', figsize=(17,5))

In [ ]:
split_date = pd.Timestamp('01-01-2014')

In [ ]:
train = df.loc[:split_date, ['Total Ontario']]
test = df.loc[split_date:, ['Total Ontario']]

In [ ]:
train.shape, test.shape

In [ ]:
ax = train.plot(figsize=(17,5))
test.plot(ax=ax)
plt.legend(['train', 'test'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

In [ ]:
train_sc[:4]

In [ ]:
X_train = train_sc[:-1]
y_train = train_sc[1:]

X_test = test_sc[:-1]
y_test = test_sc[1:]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

### Fully connected predictor

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=1, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)

In [ ]:
model.fit(X_train, y_train, epochs=200,
          batch_size=500, verbose=1,
          callbacks=[early_stop])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_test)
plt.plot(y_pred)
plt.xlim(1200,1300)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, y_pred)

## Lagged features

In [ ]:
train_sc.shape

In [ ]:
train_sc_df = pd.DataFrame(train_sc, columns=['Scaled'], index=train.index)
test_sc_df = pd.DataFrame(test_sc, columns=['Scaled'], index=test.index)
train_sc_df.head()

In [ ]:
window_size = 24

In [ ]:
for s in range(1, window_size + 1):
    train_sc_df['shift_{}'.format(s)] = train_sc_df['Scaled'].shift(s)
    test_sc_df['shift_{}'.format(s)] = test_sc_df['Scaled'].shift(s)

In [ ]:
train_sc_df.head(window_size)

In [ ]:
X_train = train_sc_df.dropna().drop('Scaled', axis=1).values
y_train = train_sc_df.dropna()[['Scaled']].values

X_test = test_sc_df.dropna().drop('Scaled', axis=1).values
y_test = test_sc_df.dropna()[['Scaled']].values

In [ ]:
X_train.shape

### Fully Connected on Windows

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=window_size, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=200,
          batch_size=500, verbose=1, callbacks=[early_stop])

In [ ]:
y_pred_fc_window = model.predict(X_test)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_test)
plt.plot(y_pred_fc_window)
plt.xlim(1200,1300)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_test)
plt.plot(y_pred_fc_window)
plt.xlim(22800,22900)

## Exercise

Reshape the input to `(num_samples, window_size, 1)`. This means we consider each input window as a sequence of `window_size` values that we will pass in sequence to the LSTM. In principle this looks like a more accurate description of our situation. But does it yield better predictions? Let's check it.

- Reshape `X_train` and `X_test` so that they represent a set of univariate sequences
- train the best recurrent model you can, you'll have to adapt the `input_shape`
- check the performance of this new model, is it better at predicting the test data?
- plot your results
- try setting the 'go_backwards=True' option
- try using GRU instead of LSTM

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
X_train_t = X_train.reshape(X_train.shape[0], window_size, 1)
X_test_t = X_test.reshape(X_test.shape[0], window_size, 1)

In [ ]:
X_train_t.shape

In [ ]:
model = Sequential()

model.add(LSTM(12, input_shape=(window_size, 1), go_backwards=True))

model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_t, y_train, epochs=100,
          batch_size=500, verbose=1, callbacks=[early_stop])

In [ ]:
y_pred_lstm_window = model.predict(X_test_t)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_test)
plt.plot(y_pred_lstm_window)
plt.xlim(1200,1300)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_test)
plt.plot(y_pred_lstm_window)
plt.xlim(22800,22900)